In [1]:
import pandas as pd
from keras.models import load_model

In [ ]:
# TODO: Set the path to the `.h5` file of the model to be loaded.
model_path = 'path/to/trained/model.h5'
model = load_model(model_path)

# 在测试集上预测

In [24]:
df_test = pd.read_csv('rank/Tests/question.csv', header=None)
df_test.columns = ['filename', 'label_name', 'label']
fnames_test = df_test.filename
n_test = len(df_test)
df_test.head()

,filename,label_name,label
0,Images/collar_design_labels/60951f5761ea3a01d6...,collar_design_labels,?
1,Images/collar_design_labels/677e1183282769a3fe...,collar_design_labels,?
2,Images/collar_design_labels/3f5a2be205e5d253eb...,collar_design_labels,?
3,Images/collar_design_labels/1712bf3d7b57d779ad...,collar_design_labels,?
4,Images/collar_design_labels/0b2b4254f35ce3a41a...,collar_design_labels,?


In [25]:
def f(index):
    return index, cv2.resize(cv2.imread('rank/'+fnames_test[index]), (width, width))

X_test = np.zeros((n_test, width, width, 3), dtype=np.uint8)
with multiprocessing.Pool(12) as pool:
    with tqdm(pool.imap_unordered(f, range(n_test)), total=n_test) as pbar:
        for i, img in pbar:
            X_test[i] = img[:,:,::-1]

100%|██████████| 10080/10080 [00:14<00:00, 682.05it/s]


In [26]:
y_pred = model.predict(X_test, batch_size=64, verbose=1)

10080/10080 [==============================] - 78s 8ms/step


In [27]:
for i in range(n_test):
    problem_name = df_test.label_name[i].replace('_labels', '')
    problem_index = label_names.index(problem_name)
    probs = y_pred[problem_index][i]
    df_test.label[i] = ';'.join(np.char.mod('%.8f', probs))

In [28]:
fname_csv = 'pred_%s.csv' % model_name
fname_zip = 'pred_%s.zip' % model_name
df_test.to_csv(fname_csv, index=None, header=None)